# Praktikum 6


Lakukan percobaan penggunaan ANNOY, FAISS, dan HNSWLIB pada dataset sekunder berukuran besar (Micro Spotify) pada link berikut: https://www.kaggle.com/datasets/bwandowando/spotify-songs-with-attributes-and-lyrics/data . Download data dan load CSV filenya (pilih dataset yg pertama dari dua dataset). pilih hanya fitur numerik saja, dan lakukan normalisasi menggunakan StandardScaler. Lakukan pencarian track terdekat dan bandingkan hasilnya.

In [ ]:
import pandas as pd
import numpy as np
import time
import faiss
from annoy import AnnoyIndex
import hnswlib
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import kagglehub

# -------------------------------
# Load dataset
# -------------------------------

print("Path to dataset files:")
df = pd.read_csv("..\data\songs_with_attributes_and_lyrics.csv")
features = ['danceability', 'energy', 'loudness', 'speechiness', 
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
X = df[features].values

# Standarisasi fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = 10  # jumlah nearest neighbors

# -------------------------------
# Exact Nearest Neighbor (brute-force)
# -------------------------------
start = time.time()
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(X_scaled)
dist_exact, idx_exact = nn.kneighbors(X_scaled)
time_exact = time.time() - start
print(f"Exact NN done in {time_exact:.3f} s")

# -------------------------------
# Annoy
# -------------------------------
start = time.time()
f = X_scaled.shape[1]
index_annoy = AnnoyIndex(f, 'euclidean')
for i, v in enumerate(X_scaled):
    index_annoy.add_item(i, v)
index_annoy.build(10)
idx_annoy = [index_annoy.get_nns_by_vector(v, k) for v in X_scaled]
time_annoy = time.time() - start
print(f"Annoy done in {time_annoy:.3f} s")

# -------------------------------
# HNSW
# -------------------------------
start = time.time()
p_hnsw = hnswlib.Index(space='l2', dim=X_scaled.shape[1])
p_hnsw.init_index(max_elements=X_scaled.shape[0], ef_construction=200, M=16)
p_hnsw.add_items(X_scaled)
p_hnsw.set_ef(200)
idx_hnsw, dist_hnsw = p_hnsw.knn_query(X_scaled, k=k)
time_hnsw = time.time() - start
print(f"HNSW done in {time_hnsw:.3f} s")

# -------------------------------
# FAISS IVF
# -------------------------------
start = time.time()
quantizer = faiss.IndexFlatL2(X_scaled.shape[1])
index_faiss = faiss.IndexIVFFlat(quantizer, X_scaled.shape[1], 100, faiss.METRIC_L2)
index_faiss.train(X_scaled)
index_faiss.add(X_scaled)
index_faiss.nprobe = 10
dist_faiss, idx_faiss = index_faiss.search(X_scaled, k)
time_faiss = time.time() - start
print(f"FAISS IVF done in {time_faiss:.3f} s")

# -------------------------------
# Contoh tampilkan top-5 neighbors dari item pertama
# -------------------------------
print("\nTop-5 neighbors for first song:")
print(f"Exact NN: {idx_exact[0][:5]}")
print(f"Annoy:    {idx_annoy[0][:5]}")
print(f"HNSW:     {idx_hnsw[0][:5]}")
print(f"FAISS:    {idx_faiss[0][:5]}")

Path to dataset files:
Exact NN done in 375.246 s
Exact NN done in 375.246 s
Annoy done in 36.000 s
Annoy done in 36.000 s
HNSW done in 26.064 s
HNSW done in 26.064 s
FAISS IVF done in 61.168 s

Top-5 neighbors for first song:
Exact NN: [     0 394553 764272 837727 749223]
Annoy:    [0, 394553, 764272, 837727, 61511]
HNSW:     [     0 394553 764272 837727 749223]
FAISS:    [     0 394553 764272 837727 749223]
FAISS IVF done in 61.168 s

Top-5 neighbors for first song:
Exact NN: [     0 394553 764272 837727 749223]
Annoy:    [0, 394553, 764272, 837727, 61511]
HNSW:     [     0 394553 764272 837727 749223]
FAISS:    [     0 394553 764272 837727 749223]


Berikut analisa terhadap code pada Praktikum 6:

### 1. Tujuan Percobaan
Praktikum 6 bertujuan membandingkan performa tiga algoritma Approximate Nearest Neighbor (ANN): Annoy, FAISS, dan HNSWLIB pada dataset sekunder besar (Micro Spotify). Dataset diambil dari Kaggle dan hanya fitur numerik yang digunakan, lalu dinormalisasi dengan StandardScaler.

### 2. Langkah-langkah Utama
- **Load Dataset:** Membaca file CSV, memilih fitur numerik, dan melakukan normalisasi.
- **Exact NN (Brute-force):** Menggunakan `NearestNeighbors` dari scikit-learn untuk mencari tetangga terdekat secara eksak.
- **Annoy:** Membuat index Annoy dengan metrik Euclidean, membangun index, dan mencari tetangga terdekat.
- **HNSWLIB:** Membuat index HNSW dengan metrik L2, membangun index, dan mencari tetangga terdekat.
- **FAISS IVF:** Membuat index FAISS IVF, melakukan training, membangun index, dan mencari tetangga terdekat.
- **Perbandingan Hasil:** Menampilkan waktu komputasi dan hasil tetangga terdekat dari item pertama untuk masing-masing algoritma.

### 3. Analisa Kode
- **Efisiensi:** 
  - Exact NN sangat akurat namun lambat untuk dataset besar.
  - Annoy dan HNSWLIB jauh lebih cepat untuk pencarian, cocok untuk skenario real-time.
  - FAISS IVF juga efisien, terutama untuk data besar dan mendukung optimasi lebih lanjut (GPU, nprobe).
- **Akurasi:** 
  - ANN (Annoy, HNSWLIB, FAISS IVF) memberikan hasil tetangga yang sangat mirip dengan Exact NN, meski kadang ada perbedaan urutan.
- **Scalability:** 
  - Semua algoritma ANN di sini mampu menangani ribuan hingga jutaan data, sedangkan Exact NN terbatas oleh memori dan waktu.
- **Praktis:** 
  - Kode sudah modular dan mudah diadaptasi untuk dataset lain atau parameter berbeda.
  - Penggunaan StandardScaler memastikan pencarian tetangga tidak bias akibat skala fitur.

### 4. Kesimpulan
Praktikum 6 menunjukkan bahwa Annoy, FAISS, dan HNSWLIB sangat efektif untuk pencarian tetangga terdekat pada dataset besar, dengan waktu komputasi jauh lebih cepat dibandingkan metode brute-force. Hasil tetangga yang ditemukan juga sangat mirip, sehingga ketiga algoritma ini layak digunakan untuk aplikasi rekomendasi seperti Spotify.

Jika ingin analisa lebih mendalam (misal: visualisasi recall, perbandingan speedup, atau analisa overlap tetangga), dapat ditambahkan sesuai kebutuhan.